In [ ]:
import pandas as pd
import GetOldTweets3 as got
from datetime import datetime
import time

In [ ]:
def date_range(start, end, n):
    f = '%Y-%m-%d'
    start = datetime.strptime(start, f)
    end = datetime.strptime(end, f)
    d = (end - start)/n
    for i in range(n):
        yield (start + d*i).strftime(f) 
    yield end.strftime(f)

In [ ]:
startdate = ''
enddate = ''
year = ''

dates = list(date_range(startdate, enddate, 3))

main_players = pd.read_csv('main_players.csv')
players = list(main_players[main_players.nfl_year == year]['player_name'].drop_duplicates())

colnames = ["id", "username", "player", "text", "date", "retweets", "favorites",
            "hashtags", "geo", "mentions", "to", "permalink"]

df = pd.DataFrame(columns = colnames)

for player in players:
    print("--- %d. Getting tweets for player %s ---" % player)
    for i in range(len(dates)-1):
        tweet_criteria = got.manager.TweetCriteria().setQuerySearch(
            "%s #fantasyfootball) lang:en" % player)\
            .setSince(dates[i])\
            .setUntil(dates[i+1])
        tweets = got.manager.TweetManager.getTweets(tweet_criteria)
        results = [[tweet.id, tweet.username, player, tweet.text, tweet.date,
                    tweet.retweets, tweet.favorites, tweet.hashtags,
                    tweet.geo,tweet.mentions, tweet.to, tweet.permalink]
                   for tweet in tweets]
        if i == 0 and len(results) <= 10:
            break
        temp = pd.DataFrame(results, columns = colnames)
        df = df.append(temp)
        print("     added %d tweets" %len(results))
        time.sleep(20)
    

df.to_csv("tweets.csv")
